### Importing relevant libraries for data imports and webscraping

In [1]:
import os
import pandas as pd
import sqlite3
from geopy.geocoders import Nominatim

#Libraries for dateime
from datetime import datetime
from datetime import timedelta

#Libraries for webscraping
import requests
import json
from bs4 import BeautifulSoup

### BIXI Data

In [27]:
folder = 'BixiRentals2018'  # instert folder name containing all bixi transactions for the year

def get_bixi_data(folder):
    """
    Returns dataframe combining all .csv files contained in supplied folder name into one dataframe
    sorted by date of transaction
    """
    
    files = []
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            files.append(pd.read_csv(f'{folder}/{file}'))

    df = pd.concat(files, axis=0, ignore_index=True)
    df = df.sort_values(by='start_date')
    
    # assign name to df to later save into a SQL table
    df.name = 'bixi'
    
    return df

In [28]:
bixi = get_bixi_data(folder)

### Bixi Stations

In [29]:
def get_stations(file):
    """
    Returns dataframe with stations with added neighborhood information
    """
    
    df = pd.read_csv(file)
    
    geolocator = Nominatim()

    df['coordinates'] = list(zip(df.latitude,df.longitude))
    df['neighborhood_'] = df.coordinates.apply(lambda x: geolocator.reverse(x).raw['address'])
    df['neighborhood'] = df.neighborhood_.apply(lambda x: x['suburb'] if 'suburb' in x.keys() else x['town'])
    df.drop(['coordinates','neighborhood_'],axis = 1, inplace = True)
    df.rename(columns = {'name':'location'}, inplace = True)
    df.name = 'stations'
    
    return df

In [30]:
stations = get_stations('Stations_2018.csv')

/Users/pacman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Using Nominatim with the default "geopy/1.19.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


GeocoderTimedOut: Service timed out

### Webscraping

#### Part 1 : Weather data from wunderground.com - April-November 2018 inclusive as BIXI is available only part of the year in Montreal.

Source: https://www.wunderground.com/history/daily/ca/montreal/CYUL/

In [37]:
lat = '45.445'
long = '-73.751'
key = '6532d6454b8aa370768e63d6ba5a832e'

def get_weather_month(lat,long,key,start,end):
    """
    Returns dataframe with weather scraped from api/weather.com for the latitude & longitude mentioned
    and start to end date. There is a maximum of 31 days per request
    """
    
    url = f'https://api.weather.com/v1/geocode/{lat}/{long}/observations/historical.json?apiKey={key}&startDate={start}&endDate={end}&units=e'
    results = requests.get(url).json()['observations']
    df = pd.DataFrame([result for result in results])
    
    # Columns to keep
    keep_columns = ['valid_time_gmt','day_ind','feels_like','heat_index','precip_hrly','precip_total',
                    'rh','temp','uv_desc','uv_index','wspd','wx_phrase']
    
    # Creates df with columns to keep
    df = df[keep_columns]
    
    # Converting Greenwich meantime timestamp into date time
    df['valid_time_gmt'] = df['valid_time_gmt'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M'))
    df['valid_time_gmt'] = pd.to_datetime(df['valid_time_gmt'])
    
    # Renaming for better understanding
    # rh stands for relative humidity and wx_phrase stands for weather condition
    df.rename(columns={'valid_time_gmt':'date_hour','rh':'humidity','wx_phrase':'condition','wspd':'wind_speed'}, inplace=True)

    return df

In [38]:
def get_all_weather(lat, long, key, year):
    """
    Returns dataframe combining weather information for supplied latitude and longitude for the months
    of April-November inclusive for the specified year
    """
    
    #Calling get_weather function for needed months
    apr = get_weather_month(lat,long,key,f'{year}0401',f'{year}0430')
    may = get_weather_month(lat,long,key,f'{year}0501',f'{year}0531')
    jun = get_weather_month(lat,long,key,f'{year}0601',f'{year}0630')
    jul = get_weather_month(lat,long,key,f'{year}0701',f'{year}0731')
    aug = get_weather_month(lat,long,key,f'{year}0801',f'{year}0831')
    sep = get_weather_month(lat,long,key,f'{year}0901',f'{year}0930')
    octo = get_weather_month(lat,long,key,f'{year}1001',f'{year}1031')
    nov = get_weather_month(lat,long,key,f'{year}1101',f'{year}1130')
    
    #Bixi is available for April to November inclusive
    months = [apr,may,jun,jul,aug,sep,octo,nov]
    
    df = pd.concat(months, axis=0, ignore_index=True)
    
    # assign name to df to later save into a SQL table
    df.name = 'weather'
    
    return df

In [39]:
weather = get_all_weather(lat,long,key,2018)

#### Part 2: Scraping public holidays

Source: https://www.officeholidays.com/countries/canada/quebec/2018.php

In [24]:
year = 2018

def get_holidays(year):
    """
    Returns dataframe with scraped holidays for the province of Quebec for the supplied year
    """

    url = f'https://www.officeholidays.com/countries/canada/quebec/{year}.php'
    html = requests.get(url)
    
    # Pass the page contents to beautiful soup for parsing
    soup = BeautifulSoup(html.content, 'html.parser') 
        
    # Get holiday dates
    holidays = ["".join(i.contents) for i in soup.find_all('time')]
    
    # Get holiday description
    holiday_desc = [i.contents for i in soup.find_all('a')][-22:-11]
    holiday_desc = [val.strip('  ') for holiday in holiday_desc for val in holiday]
    
    # Create dataframe with Holiday & Description
    df = pd.DataFrame(list(zip(holidays, holiday_desc)), columns= ['holidays','holiday_desc'])
    
    # assign name to df to later save into a SQL table
    df.name = 'holiday'
    
    return df

In [25]:
holidays = get_holidays(2018)

### Saving results to SQL database

In [40]:
dfs = [bixi, weather, stations, holidays]

In [41]:
def save_to_sql(dfs, db_name):
    """
    Creates a SQL database with the specified name and saves dataframes
    listed in dfs database
    """
    
    # Create database

    connex = sqlite3.connect(f'{db_name}.db')  
    cur = connex.cursor()

    # Create SQL tables in database
    for df in dfs:
        df.to_sql(name = df.name, con=connex, if_exists="replace", index=False)  
    
    # Close 
    connex.close()

In [42]:
save_to_sql(dfs, 'bixi_data')